# CTGAN Model

In this guide we will go through a series of steps that will let you discover
functionalities of the `CTGAN` model, including how to:

* Create an instance of `CTGAN`.
* Fit the instance to your data.
* Generate synthetic versions of your data.
* Use the a `CTGAN` to anonymize PII information.
* Customize the data tranformations to improve the learning process.
* Specify model hyperparameters to improve the output quality.

## What is CTGAN?

The `sdv.tabular.CTGAN` model from `SDV` is based on the `ctgan.CTGANSynthesizer` class
from the [CTGAN library](https://github.com/sdv-dev/CTGAN), a Deep Learning based data
synthesizer that uses Generative Adversarial Networks to generate tabular data and which
was presented at the NeurIPS 2020 conference by the paper titled [Modeling Tabular data
using Conditional GAN](https://arxiv.org/abs/1907.00503). For more details about the model,
please read the linked paper and visit the [CTGAN library](https://github.com/sdv-dev/CTGAN).

Let's now discover how to learn a dataset and later on generate synthetic data with the same
format and statistical properties by using the `CTGAN` class from SDV.

## Quick Usage

We will start by loading one of our demo datasets, the `student_placements`, which contains information
about MBA students that applied for placements during the year 2020.

<div class="alert alert-warning">

WARNING

In order to follow this guide you need to have `ctgan` installed on your system.
If you have not done it yet, please install `ctgan` now by executing the command
`pip install sdv[ctgan]` in a terminal.

</div>

In [1]:
# Setup logging and warnings - change ERROR to INFO for increased verbosity
import logging
logging.basicConfig(level=logging.ERROR)

logging.getLogger().setLevel(level=logging.ERROR)
logging.getLogger('sdv').setLevel(level=logging.ERROR)

import warnings
warnings.simplefilter("ignore")

In [2]:
from sdv.demo import load_tabular_demo

data = load_tabular_demo('student_placements')
data.head().T

,0,1,2,3,4
student_id,17264,17265,17266,17267,17268
gender,M,M,M,M,M
second_perc,67,79.33,65,56,85.8
high_perc,91,78.33,68,52,73.6
high_spec,Commerce,Science,Arts,Science,Commerce
degree_perc,58,77.48,64,52,73.3
degree_type,Sci&Tech,Sci&Tech,Comm&Mgmt,Sci&Tech,Comm&Mgmt
work_experience,False,True,False,False,False
experience_years,0,1,0,0,0
employability_perc,55,86.5,75,66,96.8


As you can see, this table contains information about students which includes, among other things:

- Their id and gender
- Their grades and specializations
- Their work experience
- The salary that they where offered
- The duration and dates of their placement

You will notice that there is data with the following characteristics:

- There are float, integer, boolean, categorical and datetime values.
- There are some variables that have missing data. In particular, all the data related to the
  placement details is missing in the rows where the studen was not placed.

Let us use `CTGAN` to learn this data and then sample synthetic data about new students
to see how well de model captures the characteristics indicated above. In order to do this you wil
need to:

- Import the `sdv.tabular.CTGAN` class and create an instance of it.
- Call its `fit` method passing our table.
- Call its `sample` method indicating the number of synthetic rows that you want to generate.

In [3]:
from sdv.tabular import CTGAN

model = CTGAN()
model.fit(data)

<div class="alert alert-info">

**NOTE**

Notice that the model `fitting` process took care of transforming the different fields using the
appropriate [Reversible Data Transforms](http://github.com/sdv-dev/RDT) to ensure that the data has
a format that the CTGANSynthesizer class can handle.

</div>

### Generate synthetic data from the model

Once the modeling has finished you are ready to generate new synthetic data by calling the `sample` method
from your model passing the number of rows that we want to generate.

In [4]:
new_data = model.sample(200)

This will return a table identical to the one which the model was fitted on, but filled with new data
which resembles the original one.

In [5]:
new_data.head()

,student_id,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,17375,M,93.536079,39.133194,Science,68.970905,Comm&Mgmt,True,0,43.519218,Mkt&Fin,58.572051,26082.597732,True,2020-03-23,NaT,NaN
1,17373,M,75.599797,47.791445,Science,79.210846,Comm&Mgmt,False,0,92.452952,Mkt&HR,75.976786,27216.544660,True,2020-01-30,2020-07-12,NaN
2,17464,M,79.000354,42.698306,Commerce,67.663749,Comm&Mgmt,False,0,54.329990,Mkt&Fin,65.469225,26588.487024,False,NaT,2020-02-11,NaN
3,17422,F,74.743617,49.965471,Commerce,83.027256,Comm&Mgmt,True,0,103.078919,Mkt&HR,58.714023,NaN,False,2020-08-26,2020-07-11,6.0
4,17499,F,82.496356,58.791322,Commerce,65.279589,Sci&Tech,False,0,55.879584,Mkt&HR,64.878618,18846.507737,False,2020-02-14,2020-07-21,NaN


<div class="alert alert-info">

**Note**

You can control the number of rows by specifying the number of `samples` in the
`model.sample(<num_rows>)`. To test, try `model.sample(10000)`. Note that the original 
table only had ~200 rows.

</div>

### Save and Load the model

In many scenarios it will be convenient to generate synthetic versions of your data
directly in systems that do not have access to the original data source. For example,
if you may want to generate testing data on the fly inside a testing environment that
does not have access to your production database. In these scenarios, fitting the
model with real data every time that you need to generate new data is feasible, so you
will need to fit a model in your production environment, save the fitted model into a
file, send this file to the testing environment and then load it there to be able to
`sample` from it.

Let's see how this process works.

#### Save and share the model

Once you have fitted the model, all you need to do is call its `save` method passing the
name of the file in which you want to save the model. Note that the extension of the filename
is not relevant, but we will be using the `.pkl` extension to highlight that the serialization
protocol used is [pickle](https://docs.python.org/3/library/pickle.html).

In [6]:
model.save('my_model.pkl')

This will have created a file called `my_model.pkl` in the same directory in which you are
running SDV.

<div class="alert alert-info">

**IMPORTANT**
    
If you inspect the generated file you will notice that its size is much smaller
than the size of the data that you used to generate it. This is because the serialized model
contains **no information about the original data**, other than the parameters it needs to
generate synthetic versions of it. This means that you can safely share this `my_model.pkl`
file without the risc of disclosing any of your real data!
    
</div>

#### Load the model and generate new data

The file you just generated can be send over to the system where the synthetic data will be
generated. Once it is there, you can load it using the `CTGAN.load` method, and
then you are ready to sample new data from the loaded instance:

In [7]:
loaded = CTGAN.load('my_model.pkl')
new_data = loaded.sample(200)

<div class="alert alert-warning">
    
**WARNING**
    
Notice that the system where the model is loaded needs to also have `sdv` and `ctgan`
installed, otherwise it will not be able to load the model and use it.
    
</div>

### Specifying the Primary Key of the table

One of the first things that you may have noticed when looking that demo data
is that there is a `student_id` column which acts as the primary key of the table,
and which is supposed to have unique values. Indeed, if we look at the number of
times that each value appears, we see that all of them appear at most once:

In [8]:
data.student_id.value_counts().max()

1

However, if we look at the synthetic data that we generated, we observe that there
are some values that appear more than once:

In [9]:
new_data.student_id.value_counts().max()

4

In [10]:
new_data[new_data.student_id == new_data.student_id.value_counts().index[0]]

,student_id,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
131,17519,M,86.162864,57.025937,Commerce,80.210574,Others,False,0,63.311080,Mkt&HR,72.817786,975.722060,True,2020-01-18,2020-07-10,NaN
159,17519,M,85.016340,72.733375,Science,54.492194,Sci&Tech,True,0,42.503027,Mkt&HR,54.286882,29388.009527,False,NaT,2020-06-01,NaN
166,17519,M,77.145421,50.919941,Science,65.318015,Comm&Mgmt,False,0,50.793900,Mkt&Fin,57.873244,15362.790626,True,NaT,2020-07-02,3.0
176,17519,F,71.330523,51.720713,Commerce,75.227234,Comm&Mgmt,False,0,56.422361,Mkt&HR,64.692054,26278.590055,True,2020-08-01,NaT,6.0


This happens because the model was not notified at any point about the fact that the
`student_id` had to be unique, so when it generates new data it will provoke collisions
sooner or later. In order to solve this, we can pass the argument `primary_key` to our
model when we create it, indicating the name of the column that is the index of the table.

In [11]:
model = CTGAN(
    primary_key='student_id'
)
model.fit(data)
new_data = model.sample(200)

As a result, the model will learn that this column must be unique and generate a unique
sequence of valures for the column:

In [12]:
new_data.head()

,student_id,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,0,F,65.227750,73.585993,Arts,77.940845,Comm&Mgmt,True,1,57.140362,Mkt&Fin,61.738792,24200.555789,False,NaT,2020-08-23,NaN
1,1,M,62.042702,54.260888,Commerce,65.665555,Comm&Mgmt,False,0,69.794563,Mkt&Fin,73.837394,29826.293192,False,NaT,2020-07-16,3.0
2,2,F,79.798332,48.911976,Science,68.366315,Comm&Mgmt,True,0,84.311197,Mkt&Fin,55.505442,17739.223866,False,2020-02-05,2020-08-14,12.0
3,3,M,44.947235,57.727061,Science,50.676930,Others,False,0,85.678003,Mkt&HR,79.731860,27207.644553,True,2020-08-14,NaT,6.0
4,4,F,51.293770,40.340629,Commerce,64.818065,Comm&Mgmt,False,0,88.700321,Mkt&HR,74.484441,28274.229026,False,2020-02-23,2021-03-03,3.0


In [13]:
new_data.student_id.value_counts().max()

1

### Anonymizing Personally Identifiable Information (PII) 

There will be many cases where the data will contain Personally Identifiable Information
which we cannot disclose. In these cases, we will want our Tabular Models to replace the
information within these fields with fake, simulated data that looks similar to the real
one but does not contain any of the original values.

Let's load a new dataset that contains a PII field, the `student_placements_pii` demo, and
try to generate synthetic versions of it that do not contain any of the PII fields.

<div class="alert alert-info">
    
**NOTE**
    
The `student_placements_pii` dataset is a modified version of the `student_placements`
dataset with one new field, `address`, which contains PII information about the students.
Notice that this additional `address` field has been simulated and does not correspond to data
from the real users.

</div>

In [14]:
data_pii = load_tabular_demo('student_placements_pii')

In [15]:
data_pii.head().T

,0,1,2,3,4
student_id,17264,17265,17266,17267,17268
address,"70304 Baker Turnpike\nEricborough, MS 15086","805 Herrera Avenue Apt. 134\nMaryview, NJ 36510","3702 Bradley Island\nNorth Victor, FL 12268",Unit 0879 Box 3878\nDPO AP 42663,"96493 Kelly Canyon Apt. 145\nEast Steven, NC 3..."
gender,M,M,M,M,M
second_perc,67,79.33,65,56,85.8
high_perc,91,78.33,68,52,73.6
high_spec,Commerce,Science,Arts,Science,Commerce
degree_perc,58,77.48,64,52,73.3
degree_type,Sci&Tech,Sci&Tech,Comm&Mgmt,Sci&Tech,Comm&Mgmt
work_experience,False,True,False,False,False
experience_years,0,1,0,0,0


If we use our tabular model on this new data we will see how the synthetic
data that it generates discloses the addresses from the real students:

In [16]:
model = CTGAN(
    primary_key='student_id',
)
model.fit(data_pii)

In [17]:
new_data_pii = model.sample(200)
new_data_pii.head()

,student_id,address,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,0,46953 William Brooks Suite 176\nNew Patrickhav...,M,68.026822,69.881979,Science,55.025418,Comm&Mgmt,True,1,86.348880,Mkt&Fin,67.727521,29670.541379,True,2020-02-18,NaT,3.0
1,1,"1729 Thomas Islands Apt. 583\nSanchezview, ND ...",F,48.329990,55.121717,Science,68.373525,Comm&Mgmt,False,0,112.312967,Mkt&HR,74.225599,29532.697993,True,NaT,2021-04-01,12.0
2,2,"PSC 2024, Box 1677\nAPO AP 99732",M,74.315719,69.674064,Commerce,72.526606,Comm&Mgmt,False,0,71.388313,Mkt&Fin,66.635816,55451.280013,False,2020-03-23,NaT,6.0
3,3,"65737 Meyer Junction Suite 154\nWest Steven, N...",M,56.654019,76.821245,Commerce,74.945972,Sci&Tech,True,1,96.431264,Mkt&Fin,63.601052,29856.891459,True,2020-03-18,2021-01-07,6.0
4,4,767 Katherine Turnpike Apt. 168\nPort Makaylav...,M,70.474516,63.725278,Science,67.247193,Sci&Tech,False,0,98.817520,Mkt&Fin,56.578200,30978.395973,True,2020-01-28,2021-01-25,3.0


In [18]:
new_data_pii.address.isin(data_pii.address).sum()

200

In order to solve this, we can pass an additional argument `anonymize_fields` to
our model when we create the instance. This `anonymize_fields` argument will need
to be a dictionary that contains:

- The name of the field that we want to anonymize.
- The category of the field that we want to use when we generate fake values for it.

The list complete list of possible categories can be seen in the [Faker Providers
](https://faker.readthedocs.io/en/master/providers.html) page, and it contains a huge
list of concepts such as:

- name
- address
- country
- city
- ssn
- credit_card_number
- credit_card_expier
- credit_card_security_code
- email
- telephone
- ...

In this case, since the field is an e-mail address, we will pass a dictionary indicating
the category `address`

In [19]:
model = CTGAN(
    primary_key='student_id',
    anonymize_fields={
        'address': 'address'
    }
)
model.fit(data_pii)

As a result, we can see how the real `address` values have been replaced by other fake
addresses that were not taken from the real data that we learned.

In [20]:
new_data_pii = model.sample(200)
new_data_pii.head()

,student_id,address,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,0,"319 Steven Ferry Suite 398\nSouth Andrew, IN 0...",M,61.165017,67.212723,Arts,52.564169,Sci&Tech,False,0,104.631737,Mkt&Fin,55.282683,28458.854256,True,2019-12-10,NaT,6.0
1,1,"9725 Morgan Orchard\nKellyberg, SD 60535",F,67.239738,36.120617,Science,60.216455,Comm&Mgmt,False,0,86.131651,Mkt&HR,58.125170,NaN,False,2020-01-15,2020-07-04,12.0
2,2,5602 Robert Motorway Suite 285\nWashingtonboro...,F,82.930408,62.291014,Science,51.097100,Comm&Mgmt,True,0,65.379276,Mkt&HR,57.874290,16192.203630,True,2020-02-13,2020-07-03,12.0
3,3,5175 Jeremiah Gardens Apt. 704\nSouth Kelliton...,F,72.742379,76.065908,Science,57.835424,Comm&Mgmt,True,0,94.155638,Mkt&HR,50.139780,NaN,True,2020-02-05,2020-09-13,NaN
4,4,"5078 Hayes Glens\nFlemingland, ME 75918",F,58.268069,77.774741,Science,59.687152,Comm&Mgmt,False,0,100.540756,Mkt&HR,49.118946,22031.384836,True,NaT,2020-07-08,3.0


In [21]:
new_data_pii.address.isin(data_pii.address).sum()

0

## Advanced Usage

Now that we have discovered the basics, let's go over a few more advanced usage examples
and see the different arguments that we can pass to our `CTGAN` Model in order to
customize it to our needs.

### CTGAN Hyperparameters

A part from the common Tabular Model arguments, `CTGAN` has a number of additional
hyperparameters that control its learning behavior and can impact on the
performance of the model, both in terms of quality of the generated data
and computationa time.

#### epochs and batch size

The first hyperparameters that we see are the `epochs` and `batch_size` arguments,
which control the number of iterations that the model will perform to optimize
its parameters, as well as the number of samples used in each step. Its default
values are `300` and `500` respectively, and `batch_size` needs to always be a
value which is multiple of `10`.

These hyperparameters have a very direct effect in time the training process lasts
but also on the performance of the data, so for new datasets, you might want to start
by setting a low value on both of them to see how long the training process takes on
your data and later on increase the number to acceptable values in order to improve
the performance.

#### log_frequency

Whether to use log frequency of categorical levels in conditional sampling. It
defaults to `True`.

This argument affects how the model processes the frequencies of the categorical
values that are used to condition the rest of the values. In some cases,
changing it to `False` could lead to better performance.

#### Neural Network dimensions

`CTGAN` has the following hyperparameters that allow you to control the
size of the different layers that compose its neural networks:

- embedding_dim (int): Size of the random sample passed to the Generator. Defaults to 128.
- gen_dim (tuple or list of ints): Size of the output samples for each one of the Residuals.
  A Resiudal Layer will be created for each one of the values provided. Defaults to (256, 256).
- dis_dim (tuple or list of ints): Size of the output samples for each one of the Discriminator
  Layers. A Linear Layer will be created for each one of the values provided. Defaults to (256, 256).

#### l2scale

The `l2scale` argument, which defaults to `1e-6`, sets the wheight Decay of the Adam Optimizer
used to optimize the Neural Networks.

#### verbose

Whether to print fit progress on stdout. Defaults to `False`.

<div class="alert alert-warning">
    
**WARNING**
    
The value that you set on the `batch_size` argument must always be
a multiple of `10`!

</div>

As an example, we will try to fit the `CTGAN` model slightly increasing the number of epochs,
reducing the `batch_size`, adding one additional layer to the models involved and using a
smaller wright decay.

Before we start, we will evaluate the qualtiy of the previously generated data using the
`sdv.evaluation.evaluate` function

In [22]:
from sdv.evaluation import evaluate

evaluate(new_data, data)

0.7542228579499172

Afterwards, we create a new instance of the `CTGAN` model with the
hyperparameter values that we want to use

In [23]:
model = CTGAN(
    primary_key='student_id',
    epochs=500,
    batch_size=100,
    gen_dim=(256, 256, 256),
    dis_dim=(256, 256, 256),
    l2scale=1e-07
)

And fit to our data.

In [24]:
model.fit(data)

Finally, we are ready to generate new data and evaluate the results.

In [25]:
new_data = model.sample(len(data))

In [26]:
new_data

,student_id,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,0,M,67.895582,45.200975,Science,57.526636,Comm&Mgmt,True,0,43.548191,Mkt&Fin,62.386567,NaN,False,NaT,2020-06-19,3.0
1,1,M,82.001293,53.168371,Science,76.619097,Sci&Tech,False,1,64.605748,Mkt&HR,63.394049,50914.262584,True,2020-02-17,2020-07-24,NaN
2,2,F,63.574181,42.933245,Science,45.209628,Sci&Tech,False,0,61.900410,Mkt&HR,59.775510,32850.072274,True,2020-03-18,NaT,NaN
3,3,F,79.705274,60.017454,Commerce,78.765692,Comm&Mgmt,False,0,101.256833,Mkt&Fin,82.326174,29506.470726,True,2020-03-09,2020-07-12,3.0
4,4,F,68.875480,46.408087,Science,65.263351,Comm&Mgmt,False,3,79.341035,Mkt&HR,49.492262,27056.058645,True,2020-02-13,2020-08-21,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,210,M,80.474335,58.706828,Science,70.891984,Comm&Mgmt,False,0,86.389603,Mkt&HR,57.328221,22999.702674,True,NaT,2020-07-23,3.0
211,211,F,44.390504,55.607717,Science,50.686976,Comm&Mgmt,False,0,95.631830,Mkt&HR,75.644778,NaN,True,NaT,2020-08-24,6.0
212,212,M,82.700221,66.903073,Science,59.405456,Comm&Mgmt,False,0,82.314847,Mkt&Fin,76.223132,41441.833928,False,2020-06-12,2020-07-18,3.0
213,213,F,74.860044,58.466668,Arts,89.010223,Comm&Mgmt,True,0,54.443907,Mkt&Fin,82.686727,26821.883004,True,2020-01-04,2020-05-15,NaN


In [27]:
from sdv.evaluation import evaluate

evaluate(new_data, data)

0.7471317423288609

As we can see, in this case these modifications changed the obtained results slightly,
but they did neither introduce dramatic changes in the performance.

### How do I specify constraints?

If you look closely at the data you may notice that some properties were
not completely captured by the model. For example, you may have seen
that sometimes the model produces an `experience_years` number greater
than `0` while also indicating that `work_experience` is `False`. These
type of properties are what we call `Constraints` and can also be
handled using `SDV`. For further details about them please visit the
[Handling Constraints](04_Handling_Constraints.ipynb) tutorial.

### Can I evaluate the Synthetic Data?

A very common question when someone starts using **SDV** to generate
synthetic data is: *\"How good is the data that I just generated?\"*

In order to answer this question, **SDV** has a collection of metrics
and tools that allow you to compare the *real* that you provided and the
*synthetic* data that you generated using **SDV** or any other tool.

You can read more about this in the [Evaluating Synthetic Data Generators](05_Evaluating_Synthetic_Data_Generators.ipynb) tutorial.